In [1]:
import os
import pandas as pd
import numpy as np
# import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

In [2]:
%store -r validation_data

config = validation_data.config.copy()
trip = validation_data.trip.copy()

In [3]:
df_trip = trip.copy()
mode_cat = {1: "1: walk",
            2: "2: bike",
            3: "3: sov",
            4: "4: hov 2",
            5: "5: hov 3+",
            6: "6: transit",
            8: "8: school bus",
            9: "9:tnc"}
pdpurp_cat = {1: "1: Work",
              2: "2: School",
              3: "3: Escort",
              4: "4: other home-based",
              5: "4: other home-based",
              6: "4: other home-based",
              7: "4: other home-based",
              8: "4: other home-based",
              9: "4: other home-based",
              10: "4: other home-based"}

df_trip['mode_label'] = df_trip['mode'].map(mode_cat)
df_trip['dpurp_label'] = df_trip['dpurp'].map(pdpurp_cat)

In [4]:
def plot_mode_choice(df:pd.DataFrame, title_name:str):
    df_plot = df.groupby(['source','dpurp_label','mode_label'])['trexpfac'].sum().reset_index()
    df_plot['percentage'] = df_plot.groupby(['source','dpurp_label'], group_keys=False)['trexpfac']. \
        apply(lambda x: x / float(x.sum()))
    
    df_plot_ct = df.groupby(['source','dpurp_label','mode_label'])['trexpfac'].count().reset_index(). \
        rename(columns={'trexpfac':'sample count'})
    df_plot = df_plot.merge(df_plot_ct, on=['source','dpurp_label','mode_label'])
    
    fig = px.bar(df_plot.sort_values(by=['dpurp_label','source']), x="mode_label", y="percentage", color="source",
                 facet_col='dpurp_label', facet_col_wrap=2,
                 barmode="group",hover_data=['sample count'],title=title_name)
    fig.update_layout(height=600, width=900, font=dict(size=11),
                      xaxis = dict(dtick = 1, categoryorder='category ascending'))
    fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
    fig.for_each_yaxis(lambda a: a.update(tickformat = ".1%"))
    fig.show()

In [5]:
len(df_trip)
# len(df_trip[df_trip['dpurp']==2])

9109127

In [6]:
plot_mode_choice(df_trip,"tour mode choice by purpose")